# Richards equation

In this tutorial we present how to solve a Richards equation with [PyGeoN](https://github.com/compgeo-mox/pygeon).  The unkwons are the velocity $q$ and the pressure $p$.

Let $\Omega=(0,2)\times(0,3)$ with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Given 
$k$ the matrix permeability, we want to solve the following problem: find $({q}, \psi)$ such that
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
K^{-1}(\psi) {q} + \nabla \psi = -\nabla z\\
\partial_t \theta (\psi) + \nabla \cdot {q} = 0
\end{array}
&\text{in } \Omega \times (0,T)
\end{array}
\right.
$$
with boundary conditions:

$$
\psi(t,x,z)=
\left\{
\begin{array}{ll}
\begin{array}{l} 
-2+2.2 \frac{t}{\Delta t_D}, \text{ on } \Gamma_{D_1}, t\leq\Delta t_D\\
0.2, \text{ on } \Gamma_{D_1}, t>\Delta t_D\\
1-z, \text{ on } \Gamma_{D_2}
\end{array}
\end{array}
\right., \qquad \nu \cdot q = 0 \text{ on } \Gamma_N \qquad \psi(0,x,z) = 1-z \text{ on } \Omega$$
and
$$
\Gamma_{D_1} = \left\{ (x,z) \in \partial \Omega \:|\: x \in [0,1] \wedge z=3  \right\},\\
\Gamma_{D_2} = \left\{ (x,z) \in \partial \Omega \:|\: x = 2 \wedge z \in [0,1]  \right\},\\
\Gamma_{D} = \Gamma_{D_1} \cup \Gamma_{D_2},\\
\Gamma_{N} = \partial \Omega \setminus \Gamma_D
$$

We present *step-by-step* how to create the grid, declare the problem data, and finally solve the problem.

### Import and parameters

In [1]:
import shutil
import os

import numpy as np
import scipy.sparse as sps
from scipy.sparse import linalg
import sympy as sp

import porepy as pp
import pygeon as pg

import time
from math import ceil, floor, log10, exp

import matplotlib.pyplot as plt

/opt/conda/lib/python3.11/site-packages/porepy/numerics/nonlinear/nonlinear_solvers.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange  # type: ignore


In [2]:
import sys
sys.path.insert(0, "/workspaces/richards/")

from richards.model_params import Model_Data
from richards.matrix_computer import Matrix_Computer

from richards.solver import Solver
from richards.solver_params import Solver_Data, Solver_Enum, Norm_Error

In [3]:
# Set the maximum number of iterations of the non-linear solver
K = 50

# L-scheme parameter
L = 3.501e-2

# Set the mesh refinment
N = 40

# Set the number of steps (excluding the initial condition)
num_steps = 9

# Simulation time length
T = num_steps/48

# Time switch conditions (for the boundary condition)
dt_D = 3/48

# Fluid density
rho = 1000

# Relative and absolute tolerances for the non-linear solver
abs_tol = 1e-5
rel_tol = 1e-5

# Domain tolerance
domain_tolerance = 1 / (10 * N)

# Output directory
output_directory = 'dual_single_stage'

In [4]:
# Van Genuchten model parameters ( relative permeability model )
theta_s = 0.396
theta_r = 0.131

alpha = 0.423

n = 2.06
K_s = 4.96e-2

m = 1 - 1/n

In [5]:
# Time step
dt   = (T-0)/num_steps

In [6]:
model_data = Model_Data(theta_r=theta_r, theta_s=theta_s, alpha=alpha, n=n, K_s=K_s, T=T, num_steps=num_steps)

### Domain preparation

In [7]:
# Prepare the domain and its mesh
subdomain = pp.StructuredTriangleGrid([2*N, 3*N], [2,3])

# Convert it to a mixed-dimensional grid
mdg = pp.meshing.subdomains_to_mdg([subdomain])

In [8]:
key = "flow"

# Collection of boundary conditions
bc_value = []
bc_essential = []

# Initial pressure
initial_pressure = []

# Discretizations for q and \psi
RT0 = pg.RT0(key)
P0  = pg.PwConstants(key)

In [9]:
# Initial pressure function
def initial_pressure_func(x): 
    return 1

In [10]:
# Fake loop to extract the grid and its data (i.e. conductivity tensor)
for subdomain, data in mdg.subdomains(return_data=True):
    # Prepare the inital pressure term by interpolating initial_pressure_func into the P0 space
    initial_pressure.append(P0.interpolate(subdomain, initial_pressure_func))
        
    # Get the boundary faces ids
    boundary_faces_indexes = subdomain.get_boundary_faces()

    # Gamma_D1 and Gamma_D2 boundary faces
    gamma_d1 = np.logical_and(subdomain.face_centers[0, :] > 0-domain_tolerance, np.logical_and(subdomain.face_centers[0, :] < 1+domain_tolerance, subdomain.face_centers[1, :] > 3-domain_tolerance))
    gamma_d2 = np.logical_and(subdomain.face_centers[0, :] > 2-domain_tolerance, np.logical_and(subdomain.face_centers[1, :] > 0-domain_tolerance, subdomain.face_centers[1, :] < 1+domain_tolerance))

    gamma_d  = np.logical_or(gamma_d1, gamma_d2)

    # Gamma_N is the remaining part of the boundary    
    gamma_n  = gamma_d.copy()
    gamma_n[boundary_faces_indexes] = np.logical_not(gamma_n[boundary_faces_indexes])
    
    # Set the initial conductivity tensor in data (the actual saved tensor does not matter at this stage)
    pp.initialize_data(subdomain, data, key, {
        "second_order_tensor": pp.SecondOrderTensor(np.ones(subdomain.num_cells)),
    })
    
    # Prepare the \hat{\psi} function
    def bc_gamma_d(x, t):
        if   x[0] > 2-domain_tolerance and x[1] > 0-domain_tolerance and x[1] < 1+domain_tolerance:
            res =  1
        elif x[1] > 3-domain_tolerance and x[0] > 0-domain_tolerance and x[0] < 1+domain_tolerance:
            res = min( 3.2, 1 + 2.2 * t / dt_D )
        else:
            res = 0
        
        return res

    # Add a lambda function that generates for each time instant the (discretized) natural boundary conditions for the problem
    bc_value.append(lambda t: - RT0.assemble_nat_bc(subdomain, lambda x: bc_gamma_d(x,t), gamma_d))

    # Set the essential boundary conditions (they will be enforced before solving the system)
    essential_pressure_dofs = np.zeros(P0.ndof(subdomain), dtype=bool)
    bc_essential = np.hstack((gamma_n, essential_pressure_dofs))

### Method

A quick recap:
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
K^{-1}(\psi) {q} + \nabla \psi = -\nabla z\\
\partial_t \theta (\psi) + \nabla \cdot {q} = 0
\end{array}
&\text{in } \Omega \times (0,T)
\end{array}
\right.
$$
becames, with the time discretization:

$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
K^{-1}(\psi^{n+1}_k) {q^{n+1}_{k+1}} + \nabla \psi^{n+1}_{k+1} = -\nabla z\\
\frac{\partial \theta(\psi_k^{n+1})}{\partial \psi} \frac{\psi_{k+1}^{n+1}}{\Delta t} + \nabla \cdot {q^{n+1}} = \frac{\partial \theta(\psi_k^{n+1})}{\partial \psi} \frac{\psi_{k}^{n+1}}{\Delta t} + \frac{\theta (\psi^{n}) - \theta (\psi^{n+1}_k)}{\Delta t} 
\end{array}
&\text{in } \Omega \times (0,T)
\end{array}
\right.
$$

In [11]:
if os.path.exists(output_directory):
    shutil.rmtree(output_directory)

In [12]:
# assemble initial solution
cp = Matrix_Computer(mdg)
initial_solution = np.zeros(cp.dof_RT0 + cp.dof_P0)
initial_solution[-cp.dof_P0:] += np.hstack(initial_pressure)

In [13]:
solver_data = Solver_Data(mdg=mdg, initial_solution=initial_solution, scheme=Solver_Enum.NEWTON, 
                          bc_essential=lambda t: bc_essential, eps_psi_abs=abs_tol, 
                          eps_psi_rel=rel_tol, max_iterations_per_step=K,   
                          output_directory=output_directory, L_Scheme_value=L, 
                          step_output_allowed=False,
                          report_name='dual', norm_error=Norm_Error.L2)

solver_data.set_rhs_vector_q(lambda t: np.hstack(list(cond(t) for cond in bc_value)))

In [14]:
solver = Solver(model_data=model_data, solver_data=solver_data)

In [15]:
start = time.time()
solver.solve()
end = time.time()

print('')
print(end - start)

Csv_Exporter: A file with name dual_NEWTON_richards_solver.csv is detected. I'll delete it
Time 0.02083


<lambdifygenerated-1>:2: RuntimeWarning: invalid value encountered in power
  return select([less(h, z),True], [0.265*(0.1699265174169*(-h + z)**2.06 + 1)**(-0.514563106796116) + 0.131,0.396], default=nan)


<lambdifygenerated-3>:2: RuntimeWarning: invalid value encountered in power
  return select([less(h, z),True], [-1.81574706110801*(-h + z)**1.06*(0.1699265174169*(-h + z)**2.06 + 1)**(-0.742718446601942)/(1 - (1 - 1.0*(0.1699265174169*(-h + z)**2.06 + 1)**(-1.0))**0.514563106796116)**2 - 7.26298824443202*(1 - 1.0*(0.1699265174169*(-h + z)**2.06 + 1)**(-1.0))**(-0.485436893203884)*(-h + z)**1.06*(0.1699265174169*(-h + z)**2.06 + 1)**(-1.74271844660194)/(1 - (1 - 1.0*(0.1699265174169*(-h + z)**2.06 + 1)**(-1.0))**0.514563106796116)**3,0], default=nan)
<lambdifygenerated-4>:2: RuntimeWarning: invalid value encountered in power
  return select([less(h, z),True], [0.0477323587424072*(-h + z)**1.06*(0.1699265174169*(-h + z)**2.06 + 1)**(-1.51456310679612),0], default=nan)
<lambdifygenerated-5>:2: RuntimeWarning: invalid value encountered in power
  return select([less(h, z),True], [0.0496*(1 - (1 - 1.0*(0.1699265174169*(-h + z)**2.06 + 1)**(-1.0))**0.514563106796116)**2*(0.1699265174169*(-h 

Iteration #001, error L2 relative psi:    0.114885927
Iteration #002, error L2 relative psi:    0.029506011
Iteration #003, error L2 relative psi:    0.002520961
Iteration #004, error L2 relative psi:    0.000072286
Iteration #005, error L2 relative psi:    0.000001997

Time 0.04167
Iteration #001, error L2 relative psi:    0.206441052
Iteration #002, error L2 relative psi:    0.047784322
Iteration #003, error L2 relative psi:    0.005483822
Iteration #004, error L2 relative psi:    0.000297785
Iteration #005, error L2 relative psi:    0.000026319

Time 0.0625
Iteration #001, error L2 relative psi:    0.320441777
Iteration #002, error L2 relative psi:    0.071682248
Iteration #003, error L2 relative psi:    0.009777887
Iteration #004, error L2 relative psi:    0.000870249
Iteration #005, error L2 relative psi:    0.000113379
Iteration #006, error L2 relative psi:    0.000013810

Time 0.08333
Iteration #001, error L2 relative psi:    0.198913295
Iteration #002, error L2 relative psi:   

KeyboardInterrupt: 